In [11]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import product
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [12]:
N_REPETITIONS = 20
N_ITERATIONS = 25

In [13]:
base_path = "data/ronin/rs_v4"
paths = {"20 EPIC": f"{base_path}/*/checkpoints/it*/*20*True.json",
         "15 EPIC": f"{base_path}/*/checkpoints/it*/*15*True.json",
         "10 EPIC": f"{base_path}/*/checkpoints/it*/*10*True.json",
         "20 LOW": f"{base_path}/*/checkpoints/it*/*20*False.json",
         "15 LOW": f"{base_path}/*/checkpoints/it*/*15*False.json",
         "10 LOW": f"{base_path}/*/checkpoints/it*/*10*False.json",

         }

for name, pattern in paths.items():
    n = len(glob.glob(pattern))
    print(
        f"For {name} there are {n}/{N_REPETITIONS*N_ITERATIONS} ready {n//N_ITERATIONS}")

For 20 EPIC there are 5/500 ready 0
For 15 EPIC there are 12/500 ready 0
For 10 EPIC there are 9/500 ready 0
For 20 LOW there are 10/500 ready 0
For 15 LOW there are 10/500 ready 0
For 10 LOW there are 6/500 ready 0


In [14]:
def process_df(df):
    record_df = pd.json_normalize(df['records'])
    # concatenate records data
    df = pd.concat([df, record_df],  axis=1)

    # drop original column
    df = df.drop(['records', 'index'], axis=1)

    # drop infractions columns

    # remove prefixes from column name
    df.columns = df.columns.str.removeprefix('meta.')
    df.columns = df.columns.str.removeprefix('scores.')
    df['driving_score'] = df['score_composed'] / 100
    # df = df.rename(columns={"index": "route_index"})
    df = df.set_index(['fps', 'highquality', 'rep', 'it', 'route_id'])
    return df

In [15]:

def make_df(path: str):

    # DATA FROM PATH
    data = {}
    data['path'] = path
    stem = Path(path).stem.split("_")
    data['fps'] = int(stem[1])
    data['highquality'] = stem[3]
    match path.split("/"):
        case ["data", "ronin", "rs_v2", rep, "checkpoints", it, *objects]:
            data['rep'] = int(rep)
            data['it'] = int(it[2:])
    # print(data)

    # READ CHECKPOINT
    with open(path, "r") as f:
        content = json.load(f)

        data['records'] = content['_checkpoint']['records']
        if not data['records']:
            return

        df = pd.DataFrame(data)
        return process_df(df)


dfs = []
pattern = f"{base_path}/*/checkpoints/it*/*.json"
for file in glob.glob(pattern):
    dfs.append(make_df(file))

KeyError: "None of ['rep', 'it'] are in the columns"

In [ ]:
df = pd.concat(dfs).sort_index()
# df.to_csv("df.csv")
# df[['driving_score', 'duration_system']].reset_index()['route_id'].value_counts()
df

path                    status                      infractions.collisions_layout infractions.collisions_pedestrian                     infractions.collisions_vehicle                    infractions.outside_route_lanes infractions.red_light infractions.route_dev infractions.route_timeout infractions.stop_infraction infractions.vehicle_blocked  duration_game  duration_system  route_length  score_composed  score_penalty  score_route  driving_score
fps highquality rep it route_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
10  True        0   0  RouteScenario_88   data/ronin/rs_v2/0/checkpoints/it0/fps_10_high...  Failed - Agent timed out  [Agent collided against object with type=traff...                                []  [Agent collided against object with type=vehic...                                                 []                    []                    []          [Route timeout.]                          []                          []     209.100003       550.858677    241.766581       15.519084       0.390000    39.792523       0.155191
                    1  RouteScenario_744  data/ronin/rs_v2/0/checkpoints/it1/fps_10_high...  Failed - Agent timed out                                                 []                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...                    []                    []          [Route timeout.]                          []                          []     248.100004       589.405817    290.027185       22.741855       0.553186    41.110699       0.227419
                    2  RouteScenario_718  data/ronin/rs_v2/0/checkpoints/it2/fps_10_high...  Failed - Agent timed out                                                 []                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...                    []                    []          [Route timeout.]                          []                          []     208.100003       534.188294    248.073006       31.149151       0.579327    53.767854       0.311492
                    3  RouteScenario_317  data/ronin/rs_v2/0/checkpoints/it3/fps_10_high...  Failed - Agent timed out                                                 []                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...                    []                    []          [Route timeout.]                          []                          []     464.100007      1253.714966    240.570289       32.211586       0.578287    55.701704       0.322116
                    4  RouteScenario_611  data/ronin/rs_v2/0/checkpoints/it4/fps_10_high...  Failed - Agent timed out                                                 []                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...                    []                    []          [Route timeout.]                          []                          []     249.100004       586.623461    281.715300       15.045688       0.551864    27.263378       0.150457
...                                                                                     ...                       ...                                                ...                               ...                                                ...                                                ...                   ...         

In [ ]:
# Get an index of minimal driving score for each fidelity value for each repetition
solutions_idx = df.groupby(
    ['fps', 'highquality', 'rep'],)['driving_score'].idxmin()
solutions = df.loc[solutions_idx][['driving_score', 'duration_system']]
solutions

driving_score  duration_system
fps highquality rep it route_id                                         
10  True        0   17 RouteScenario_311       0.075450       494.331251
                1   3  RouteScenario_337       0.059218       714.505364
                2   14 RouteScenario_278       0.054614      1497.857111
                3   17 RouteScenario_97        0.045128       412.859133
                4   15 RouteScenario_276       0.028958       483.226898
                5   20 RouteScenario_286       0.043634       492.360416
                6   21 RouteScenario_105       0.054533       503.102671
                7   7  RouteScenario_233       0.019790       625.858572
                8   10 RouteScenario_263       0.050729       722.696556
                9   18 RouteScenario_419       0.014509       479.442012
12  True        0   14 RouteScenario_59        0.033340       382.800361
                1   5  RouteScenario_595       0.070324       411.403101
                2   16 RouteScenario_99        0.031486       392.530027
                3   1  RouteScenario_337       0.044889       484.269693
                4   22 RouteScenario_53        0.044712       607.859215
                5   8  RouteScenario_337       0.041989       482.935943
                6   10 RouteScenario_107       0.030328       441.165970
                7   2  RouteScenario_263       0.052303       723.949086
                8   1  RouteScenario_602       0.113661       716.999831
                9   1  RouteScenario_69        0.099754       523.423729
14  True        0   8  RouteScenario_141       0.037605      3069.932215
                1   6  RouteScenario_342       0.033464       995.907285
                2   21 RouteScenario_248       0.008072       511.546673
                3   18 RouteScenario_392       0.004505       504.128639
                4   23 RouteScenario_248       0.008072       499.955741
                5   17 RouteScenario_395       0.066556       487.387737
                6   12 RouteScenario_77        0.183056       624.072903
                7   14 RouteScenario_160       0.047564       515.268482
                8   1  RouteScenario_263       0.050729       837.095898
                9   15 RouteScenario_574       0.055978       503.458481
15  True        0   20 RouteScenario_342       0.036013      1084.914932
                1   0  RouteScenario_408       0.169779      1000.179747
                2   18 RouteScenario_273       0.059833       974.183122
                3   24 RouteScenario_488       0.116338      3959.351618
                4   4  RouteScenario_338       0.154492      1072.434280
                5   13 RouteScenario_310       0.066807       951.577569
                6   15 RouteScenario_294       0.199386      1177.891253
                7   3  RouteScenario_245       0.039955      1937.514754
                8   2  RouteScenario_245       0.066194      1507.751883
                9   11 RouteScenario_337       0.125189      1089.106612
17  True        0   0  RouteScenario_155       0.117861      2443.535655
                1   3  RouteScenario_247       0.025977       872.051950
                2   14 RouteScenario_214       0.089366       629.804895
                3   4  RouteScenario_245       0.110324      1161.978930
                4   16 RouteScenario_140       0.065672      6174.141449
                5   17 RouteScenario_159       0.143719       683.299481
                6   17 RouteScenario_140       0.303804      5721.995005
                7   10 RouteScenario_231       0.127177      1092.712209
                8   17 RouteScenario_248       0.007640       645.206162
                9   22 RouteScenario_136       0.209695      2153.489664
18  True        0   13 RouteScenario_392       0.004505       635.618093
                1   2  RouteScenario_160       0.047564       641.234964
                2   2  RouteScenario_595       0.070324       585.220800
                3   17 Rout

In [ ]:
# extract route id
solutions_idx = solutions_idx.apply(lambda x: x[-1].split("_")[-1])

In [ ]:


solutions_idx = solutions_idx.rename("scenario_id")

solutions_idx.groupby('rep').apply(set).apply(lambda x: ",".join(x))

rep
0     59,589,392,141,342,155,311
1    408,388,337,595,160,342,247
2     248,214,273,297,99,278,595
3     310,488,97,392,245,337,572
4     248,140,53,276,388,338,160
5    310,395,286,258,159,337,391
6     77,140,294,159,507,107,105
7    263,408,231,233,245,160,271
8        248,263,602,261,245,271
9     69,136,407,337,730,574,419
Name: scenario_id, dtype: object

^ should be less 70, 10 for each fidelity value